In [1]:
import pandas as pd

from carla import OnlineCatalog, MLModelCatalog
from carla.recourse_methods import GrowingSpheres, Wachter, Dice, CCHVAE, Revise
from carla import Benchmark
from carla.evaluation.catalog import SuccessRate, Distance, Redundancy, AvgTime, ConstraintViolation, YNN

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[WARNING] From c:\sgrondin\Explicability\CARLA\CARLA\carla\recourse_methods\autoencoder\models\autoencoder.py:12: The name tf.disable_eager_execution is deprecated. Please use tf.compat.v1.disable_eager_execution instead.
 [module_wrapper.py _tfmw_add_deprecation_warning]


1. Dataset and model importation

In [2]:
dataset = OnlineCatalog("adult")

In [3]:
model = MLModelCatalog(dataset, "ann", backend="pytorch")

2. Generate a Counterfactual

In [4]:
dice = Dice(model, {})
factuals = dataset.df.sample(1)
cf_dice = dice.get_counterfactuals(factuals)

  0%|          | 0/1 [00:00<?, ?it/s]c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0' has dtype incompatible with f

In [5]:
factuals_decoded = dataset.inverse_transform(factuals)
cf_dice_decoded = dataset.inverse_transform(cf_dice)
factual_row = factuals_decoded.iloc[0]
cf_row = cf_dice_decoded.iloc[0]

df_pair = pd.DataFrame({
    "Factual": factual_row,
    "Counterfactual": cf_row
})

def highlight_diff(row):
    if row.iloc[-1] != row.iloc[-2]:
        return ['background-color: blue'] * len(row)
    else:
        return [''] * len(row)

styled_df = df_pair.style.apply(highlight_diff, axis=1)


styled_df


,Factual,Counterfactual
age,41.000000,55.671752
capital-gain,15024.000000,15024.000000
capital-loss,0.000000,0.000000
education-num,14.000000,14.000000
fnlwgt,101593.000000,101593.000000
hours-per-week,50.000000,50.000000
income,1,nan
marital-status,Married,Married
native-country,US,US
occupation,Managerial-Specialist,Managerial-Specialist


3. Evaluate the generate cf: 

In [6]:
benchmark_dice = Benchmark(model, dice, factuals, cf_dice)
benchmark_dice.run_benchmark(measures=[SuccessRate(), Distance(model), YNN(model, {"y": 5, "cf_label": 1}),  Redundancy(model, {"cf_label": 1}),  ConstraintViolation(model), AvgTime(dice)])

c:\sgrondin\Explicability\CARLA\CARLA\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
c:\sgrondin\Explicability\CARLA\CARLA\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
c:\sgrondin\Explicability\CARLA\CARLA\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: 

,Success_Rate,L0_distance,L1_distance,L2_distance,Linf_distance,y-Nearest-Neighbours,Redundancy,Constraint_Violation,avg_time
0,1.0,2.0,1.200983,1.040394,1.0,1.0,2.0,2.0,0.129276


Benchmark de plusieurs méthodes: 

In [1]:
import pandas as pd
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.recourse_methods import GrowingSpheres, Wachter, Dice, CCHVAE, Revise
import carla.evaluation.catalog as evaluation_catalog
from carla.evaluation import Benchmark

dataset = OnlineCatalog("adult")  # ou "compas", "give_me_some_credit"
mlmodel = MLModelCatalog(dataset, "ann", backend="pytorch")  # ici ANN Pytorch

config_cchvae = {
    "data_name": "adult",
    "n_search_samples": 100,
    "p_norm": 1,
    "step": 0.1,
    "max_iter": 1000,
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}

config_revise = {
    "data_name": "adult",
    "lambda": 0.5,
    "optimizer": "adam",
    "lr": 0.1,
    "max_iter": 1500,
    "target_class": [0, 1],
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "activFun": None,
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}


recourse_methods = {
    "GrowingSpheres": GrowingSpheres(mlmodel),
    #"Wachter": Wachter(mlmodel),
    "Dice": Dice(mlmodel),
    #"CCHVAE": CCHVAE(mlmodel, config_cchvae),
    #"Revise": Revise(mlmodel, dataset, config_revise),
}
n_instances = 10
factuals = dataset.df.sample(10)



all_results = []

for name, recourse_method in recourse_methods.items():
    benchmark = Benchmark(mlmodel, recourse_method, factuals)
    metrics = [
    evaluation_catalog.YNN(mlmodel),
    evaluation_catalog.Distance(mlmodel),
    evaluation_catalog.SuccessRate(),
    evaluation_catalog.Redundancy(mlmodel),
    evaluation_catalog.ConstraintViolation(mlmodel),
    evaluation_catalog.AvgTime(recourse_method),  
    ]
    result = benchmark.run_benchmark(metrics)
    result["Recourse_Method"] = name
    all_results.append(result)

results_df = pd.concat(all_results, ignore_index=True)
results_df.set_index("Recourse_Method", inplace=True)
results_df


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[WARNING] From c:\sgrondin\Explicability\CARLA\CARLA\carla\recourse_methods\autoencoder\models\autoencoder.py:12: The name tf.disable_eager_execution is deprecated. Please use tf.compat.v1.disable_eager_execution instead.
 [module_wrapper.py _tfmw_add_deprecation_warning]


c:\sgrondin\Explicability\CARLA\CARLA\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
c:\sgrondin\Explicability\CARLA\CARLA\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
c:\sgrondin\Explicability\CARLA\CARLA\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: 

,y-Nearest-Neighbours,L0_distance,L1_distance,L2_distance,Linf_distance,Success_Rate,Redundancy,Constraint_Violation,avg_time
Recourse_Method,,,,,,,,,
GrowingSpheres,0.266667,5.777778,1.024311,0.797980,0.699460,0.9,3.666667,0.0,0.003263
Dice,0.600000,1.700000,1.085207,0.882411,0.767117,1.0,0.800000,1.1,0.137430


In [ ]:
import pandas as pd
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.recourse_methods import GrowingSpheres, Wachter, Dice, CCHVAE, Revise
import carla.evaluation.catalog as evaluation_catalog
from carla.evaluation import Benchmark
from carla.models.negative_instances import predict_negative_instances

dataset = OnlineCatalog("adult")  # ou "compas", "give_me_some_credit"
mlmodel = MLModelCatalog(dataset, "ann", backend="pytorch")  # ici ANN Pytorch

config_cchvae = {
    "data_name": "adult",
    "n_search_samples": 100,
    "p_norm": 1,
    "step": 0.1,
    "max_iter": 1000,
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}

config_revise = {
    "data_name": "adult",
    "lambda": 0.5,
    "optimizer": "adam",
    "lr": 0.1,
    "max_iter": 1500,
    "target_class": [0, 1],
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "activFun": None,
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}


recourse_methods = {
    "GrowingSpheres": GrowingSpheres(mlmodel),
    "Wachter": Wachter(mlmodel),
    "Dice": Dice(mlmodel),
    "CCHVAE": CCHVAE(mlmodel, config_cchvae),
    "Revise": Revise(mlmodel, dataset, config_revise),
}
n_instances = 10
factuals = dataset.df.sample(10)

metrics = [
    evaluation_catalog.YNN(mlmodel, {"y": 5, "cf_label": 1}),
    evaluation_catalog.Distance(mlmodel),
    evaluation_catalog.SuccessRate(),
    evaluation_catalog.Redundancy(mlmodel, {"cf_label": 1}),
    evaluation_catalog.ConstraintViolation(mlmodel),
    #evaluation_catalog.AvgTime({}),  # ou avec {"time": ...} si timer dispo
]

all_results = []

for name, recourse_method in recourse_methods.items():
    benchmark = Benchmark(mlmodel, recourse_method, factuals)
    result = benchmark.run_benchmark(metrics)
    result["Recourse_Method"] = name
    all_results.append(result)

results_df = pd.concat(all_results, ignore_index=True)
results_df.set_index("Recourse_Method", inplace=True)
results_df


[INFO] Start training of Variational Autoencoder... [vae.py fit]
[INFO] [Epoch: 0/5] [objective: 76.530] [vae.py fit]
[INFO] [ELBO train: 76.53] [vae.py fit]
[INFO] [ELBO train: 87.51] [vae.py fit]
[INFO] [ELBO train: 94.33] [vae.py fit]
[INFO] [ELBO train: 101.45] [vae.py fit]
[INFO] [ELBO train: 108.32] [vae.py fit]
[INFO] ... finished training of Variational Autoencoder. [vae.py fit]
[INFO] Start training of Variational Autoencoder... [vae.py fit]
[INFO] [Epoch: 0/5] [objective: 76.148] [vae.py fit]
[INFO] [ELBO train: 76.15] [vae.py fit]
[INFO] [ELBO train: 87.03] [vae.py fit]
[INFO] [ELBO train: 94.4] [vae.py fit]
[INFO] [ELBO train: 101.56] [vae.py fit]
[INFO] [ELBO train: 108.35] [vae.py fit]
[INFO] ... finished training of Variational Autoencoder. [vae.py fit]


/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/pytho

KeyError: "['Distance_1', 'Distance_2', 'Distance_3', 'Distance_4'] not in index"